In [29]:
import numpy as np
from scipy.linalg import expm
from matplotlib import pyplot as plt
import cirq
from cirq import H, CNOT, X

In [3]:
ident = np.array([[1., 0.], [0., 1.]])
sigma_x = np.array([[0., 1.], [1., 0.]])
sigma_y = np.array([[0., -1.0j], [1.0j, 0.]])
sigma_z = np.array([[1., 0.], [0., -1.]])

In [22]:
# Random angles correctly sampled for the Haar measure

def omega():
    return np.random.rand() * np.pi * 2
def phi ():
    return np.random.rand() * np.pi * 2
def theta():
    return np.arccos(2 * np.random.rand() - 1)

def unitary_angles(n_qubits):
    omegas = [omega() for i in range(n_qubits)]
    phis = [phi() for i in range(n_qubits)]
    thetas = [theta() for i in range(n_qubits)]

    return [phis,omegas,thetas]

# Unitary matrix constructed from three angles

def unitary(phi,omega,theta):
    
    return expm(-0.5j * omega * sigma_z) @ expm(-0.5j * theta * sigma_x) @ expm(-0.5j * phi * sigma_z)

# Projectors onto |0> and |1> states for an invidual qubit
 
projectors = np.zeros([2,2,2])
projectors[0,0,0] = 1
projectors[1,1,1] = 1

In [24]:
# Given an n-qubit circuit, attach the relevant random unitaries and measurements

def shadows(qubits,circuit,angles,n_qubits):          # applies unitaries and measurement to first n qubits (i.e. put ancillas at the end)

    shadows = cirq.Circuit()

    # Attach circuit of interest

    shadows.append(circuit)


    # Random unitaries nitaries

    shadows.append([cirq.rz(angles[0][i])(qubits[i]) for i in range(n_qubits)],strategy=cirq.circuits.InsertStrategy.NEW_THEN_INLINE)
    shadows.append([cirq.rx(angles[2][i])(qubits[i]) for i in range(n_qubits)],strategy=cirq.circuits.InsertStrategy.NEW_THEN_INLINE)
    shadows.append([cirq.rz(angles[1][i])(qubits[i]) for i in range(n_qubits)],strategy=cirq.circuits.InsertStrategy.NEW_THEN_INLINE)

    # Measuring

    shadows.append(cirq.measure(*qubits[0:n_qubits],key='out'))

    return shadows

# Example: GHZ circuit

GHZ_qubits = cirq.LineQubit.range(3)

GHZ_circuit = cirq.Circuit(cirq.H(GHZ_qubits[0]),
                           cirq.CNOT(GHZ_qubits[0],GHZ_qubits[1]),
                           cirq.CNOT(GHZ_qubits[1],GHZ_qubits[2]))

print(shadows(GHZ_qubits,GHZ_circuit,unitary_angles(3),3))


0: ───H───@───────Rz(1.66π)────Rx(0.845π)───Rz(0.469π)───M('out')───
          │                                              │
1: ───────X───@───Rz(1.11π)────Rx(0.707π)───Rz(1.44π)────M──────────
              │                                          │
2: ───────────X───Rz(0.246π)───Rx(0.374π)───Rz(0.284π)───M──────────


In [30]:
# More interesting example, the three edge setup with an ancilla and two plaquette defects

fusion_qubits = cirq.LineQubit.range(11)
g1_m, g1_R2, g1_R, g2_m, g2_R2, g2_R, g3_m, g3_R2, g3_R, x1, x2 = fusion_qubits

fusion = cirq.Circuit()

# Prepare ground state
fusion.append([
    H(g1_m),
    H(g1_R2),
    H(g2_R)
])

fusion.append([
    CNOT(g1_m,g2_m),
    CNOT(g1_m,g3_m),
    CNOT(g1_R2,g2_R2),
    CNOT(g1_R2,g3_R2),
    CNOT(g1_R,g2_R),
    CNOT(g1_R,g3_R),
])

# Prepare the ancilla in the superposition of the conjugacy class and multiply onto g1 and g2

fusion.append([
    H(x1),
    CNOT(x1,x2)
])

fusion.append([
    CNOT(x1,g1_R2),
    X(g1_m),

    CNOT(x2,g2_R2),
    X(g2_m)
])

In [28]:
# Reconstruct the 'snapshot' of the density matrix using unitary matrices

def snapshot(angles,results):
    
    rho = np.eye(1)

    for i_mmt, mmt in enumerate(results):
        U = unitary(angles[0][i_mmt],angles[1][i_mmt],angles[2][i_mmt])
        
        rho = np.kron(rho, 3 * np.conj((U.T)) @ projectors[:,:,mmt] @ U - np.eye(2))
    
    return rho

# Approximate the density matrix by averaging over snapshots

n_qubits = 3

qubits = GHZ_qubits
circuit = GHZ_circuit

N_rep = 500


rho = np.zeros([2**n_qubits,2**n_qubits],dtype='complex')

for repetitions in range(N_rep):
    angles = unitary_angles(n_qubits)
    
    shadow_circuit = shadows(qubits,circuit,angles,n_qubits)

    simulator = cirq.Simulator()
    result = simulator.run(shadow_circuit).measurements['out'][0]

    rho += snapshot(angles,result)

rho /= N_rep

# expected density matrix for 3 qubit GHZ
rho_ex = np.zeros([8,8])
rho_ex[0,0] = 1/2
rho_ex[7,7] = 1/2
rho_ex[0,7] = 1/2
rho_ex[7,0] = 1/2

print(np.trace(np.conj((rho-rho_ex).T)@(rho - rho_ex)))



(0.027071599440243444+0j)
